In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

DATA_SILVER = Path("data/silver")
DATA_GOLD = Path("data/gold")
DATA_GOLD.mkdir(parents=True, exist_ok=True)

In [2]:
pokemon = pd.read_csv(DATA_SILVER / "pokemon.csv")
stats = pd.read_csv(DATA_SILVER / "pokemon_stats.csv")
types = pd.read_csv(DATA_SILVER / "pokemon_types.csv")
moves = pd.read_csv(DATA_SILVER / "moves.csv")
pokemon_moves = pd.read_csv(DATA_SILVER / "pokemon_moves.csv")
type_eff = pd.read_csv(DATA_SILVER / "type_effectiveness.csv")

pokemon.shape, stats.shape, types.shape, moves.shape, pokemon_moves.shape, type_eff.shape

((1328, 6), (7968, 4), (2077, 3), (937, 7), (95628, 3), (441, 3))

In [5]:
stats_pivot = (
    stats
    .pivot_table(
        index="pokemon_id",
        columns="stat_name",
        values="stat_value",
        aggfunc="first"
    )
    .reset_index()
)

# Aseguramos nombres de columnas limpios
stats_pivot.columns.name = None
stats_pivot.head()

,pokemon_id,attack,defense,hp,special-attack,special-defense,speed
0,1,49,49,45,65,65,45
1,2,62,63,60,80,80,60
2,3,82,83,80,100,100,80
3,4,52,43,39,60,50,65
4,5,64,58,58,80,65,80


In [6]:
# Aseguramos orden por slot
types_sorted = types.sort_values(["pokemon_id", "slot"])

# Pivot: fila por pokemon_id, columnas por slot
types_pivot = (
    types_sorted
    .pivot_table(
        index="pokemon_id",
        columns="slot",
        values="type",
        aggfunc="first"
    )
    .reset_index()
)

# Renombramos columnas a type_1, type_2
col_rename = {}
for c in types_pivot.columns:
    if isinstance(c, (int, float)):
        col_rename[c] = f"type_{int(c)}"
types_pivot = types_pivot.rename(columns=col_rename)

types_pivot.head()

slot,pokemon_id,type_1,type_2
0,1,grass,poison
1,2,grass,poison
2,3,grass,poison
3,4,fire,NaN
4,5,fire,NaN


In [7]:
# Unimos pokemon_moves con moves para tener power, type, damage_class
pm_full = (
    pokemon_moves
    .merge(moves, left_on="move_id", right_on="id", how="left", suffixes=("", "_move"))
)

pm_full.head()

,pokemon_id,move_id,move_name,id,name,type,damage_class,power,accuracy,pp
0,1,13,razor-wind,13,razor-wind,normal,special,80.0,100.0,10.0
1,1,14,swords-dance,14,swords-dance,normal,status,NaN,NaN,20.0
2,1,15,cut,15,cut,normal,physical,50.0,95.0,30.0
3,1,20,bind,20,bind,normal,physical,15.0,85.0,20.0
4,1,22,vine-whip,22,vine-whip,grass,physical,45.0,100.0,25.0


In [8]:
# Filter movimientos con power disponible (algunos status no tienen power)
pm_with_power = pm_full[pm_full["power"].notna()].copy()

# Agregados generales por Pokémon
move_aggs = pm_with_power.groupby("pokemon_id").agg(
    n_moves=("move_id", "nunique"),
    max_move_power=("power", "max"),
    mean_move_power=("power", "mean"),
)

# Agregados por damage_class
physical = pm_with_power[pm_with_power["damage_class"] == "physical"]
special = pm_with_power[pm_with_power["damage_class"] == "special"]

physical_aggs = physical.groupby("pokemon_id").agg(
    max_power_physical=("power", "max")
)
special_aggs = special.groupby("pokemon_id").agg(
    max_power_special=("power", "max")
)

# Combinamos
moves_features = (
    move_aggs
    .join(physical_aggs, how="left")
    .join(special_aggs, how="left")
    .reset_index()
)

moves_features.head()

,pokemon_id,n_moves,max_move_power,mean_move_power,max_power_physical,max_power_special
0,1,44,130.0,67.954545,130.0,130.0
1,2,41,130.0,66.219512,120.0,130.0
2,3,52,150.0,73.076923,150.0,150.0
3,4,68,150.0,72.397059,150.0,130.0
4,5,66,150.0,72.772727,150.0,130.0


In [9]:
offensive_by_type = (
    type_eff
    .groupby("attacking_type")["multiplier"]
    .mean()
    .rename("offensive_effectiveness_type")
    .reset_index()
)

offensive_by_type.head()

,attacking_type,offensive_effectiveness_type
0,bug,0.976190
1,dark,1.023810
2,dragon,0.976190
3,electric,0.976190
4,fairy,1.071429


In [10]:
# Unimos tipos del Pokémon con la tabla de efectividad ofensiva
types_off = types_pivot.copy()

# type_1
types_off = types_off.merge(
    offensive_by_type,
    left_on="type_1",
    right_on="attacking_type",
    how="left"
).rename(columns={"offensive_effectiveness_type": "off_eff_type_1"}).drop(columns=["attacking_type"])

# type_2 (puede ser NaN)
types_off = types_off.merge(
    offensive_by_type,
    left_on="type_2",
    right_on="attacking_type",
    how="left"
).rename(columns={"offensive_effectiveness_type": "off_eff_type_2"}).drop(columns=["attacking_type"])

# Expected offensive effectiveness por Pokémon
types_off["offensive_effectiveness"] = types_off[["off_eff_type_1", "off_eff_type_2"]].mean(axis=1)

types_off[["pokemon_id", "type_1", "type_2", "off_eff_type_1", "off_eff_type_2", "offensive_effectiveness"]].head()

,pokemon_id,type_1,type_2,off_eff_type_1,off_eff_type_2,offensive_effectiveness
0,1,grass,poison,0.976190,0.952381,0.964286
1,2,grass,poison,0.976190,0.952381,0.964286
2,3,grass,poison,0.976190,0.952381,0.964286
3,4,fire,NaN,1.095238,NaN,1.095238
4,5,fire,NaN,1.095238,NaN,1.095238


In [11]:
# Creamos una tabla pivote attackers x defenders para lookup rápido
eff_matrix = type_eff.pivot_table(
    index="attacking_type",
    columns="defending_type",
    values="multiplier",
    aggfunc="first"
)

eff_matrix.head()

defending_type,bug,dark,dragon,electric,fairy,fighting,fire,flying,ghost,grass,ground,ice,normal,poison,psychic,rock,shadow,steel,stellar,unknown,water
attacking_type,,,,,,,,,,,,,,,,,,,,,
bug,1.0,2.0,1.0,1.0,0.5,0.5,0.5,0.5,0.5,2.0,1.0,1.0,1.0,0.5,2.0,1.0,1.0,0.5,1.0,1.0,1.0
dark,1.0,0.5,1.0,1.0,0.5,0.5,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
dragon,1.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0
electric,1.0,1.0,0.5,0.5,1.0,1.0,1.0,2.0,1.0,0.5,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
fairy,1.0,2.0,2.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,0.5,1.0,1.0,1.0


In [12]:
def compute_defensive_score(row, eff_mat):
    t1 = row["type_1"]
    t2 = row["type_2"]

    # Si tipo1 no está en la matriz por error, devolvemos NaN
    # (en práctica, no debería ocurrir)
    attacking_types = eff_mat.index

    multipliers = []

    for atk in attacking_types:
        try:
            m1 = eff_mat.loc[atk, t1] if t1 in eff_mat.columns else 1.0
        except KeyError:
            m1 = 1.0

        if pd.notna(t2):
            try:
                m2 = eff_mat.loc[atk, t2] if t2 in eff_mat.columns else 1.0
            except KeyError:
                m2 = 1.0
            mult = m1 * m2  # combinación de tipos
        else:
            mult = m1

        multipliers.append(mult)

    if len(multipliers) == 0:
        return np.nan
    return np.mean(multipliers)


types_def = types_off.copy()
types_def["defensive_multiplier_mean"] = types_def.apply(
    compute_defensive_score, axis=1, eff_mat=eff_matrix
)

types_def[["pokemon_id", "type_1", "type_2", "defensive_multiplier_mean"]].head()

,pokemon_id,type_1,type_2,defensive_multiplier_mean
0,1,grass,poison,1.059524
1,2,grass,poison,1.059524
2,3,grass,poison,1.059524
3,4,fire,NaN,1.000000
4,5,fire,NaN,1.000000


In [13]:
features = (
    pokemon
    .merge(stats_pivot, left_on="id", right_on="pokemon_id", how="left")
    .merge(types_def[["pokemon_id", "type_1", "type_2", "offensive_effectiveness", "defensive_multiplier_mean"]],
           on="pokemon_id", how="left")
    .merge(moves_features, on="pokemon_id", how="left")
)

# Limpiamos columnas duplicadas de ids
features = features.drop(columns=["pokemon_id"])
features.head()

,id,name,height,weight,base_experience,total_base_stats,attack,defense,hp,special-attack,special-defense,speed,type_1,type_2,offensive_effectiveness,defensive_multiplier_mean,n_moves,max_move_power,mean_move_power,max_power_physical,max_power_special
0,1,bulbasaur,7,69,64.0,318,49,49,45,65,65,45,grass,poison,0.964286,1.059524,44.0,130.0,67.954545,130.0,130.0
1,2,ivysaur,10,130,142.0,405,62,63,60,80,80,60,grass,poison,0.964286,1.059524,41.0,130.0,66.219512,120.0,130.0
2,3,venusaur,20,1000,236.0,525,82,83,80,100,100,80,grass,poison,0.964286,1.059524,52.0,150.0,73.076923,150.0,150.0
3,4,charmander,6,85,62.0,309,52,43,39,60,50,65,fire,NaN,1.095238,1.000000,68.0,150.0,72.397059,150.0,130.0
4,5,charmeleon,11,190,142.0,405,64,58,58,80,65,80,fire,NaN,1.095238,1.000000,66.0,150.0,72.772727,150.0,130.0


In [14]:
# Evitar divisiones por cero
features["defensive_multiplier_mean_clipped"] = features["defensive_multiplier_mean"].clip(lower=0.1)

# Score defensivo básico: (hp + def + sp_def) / multiplier
features["defensive_raw"] = (
    features["hp"].fillna(0)
    + features["defense"].fillna(0)
    + features["special-defense"].fillna(0)
) / features["defensive_multiplier_mean_clipped"]

# Score ofensivo básico: total_base_stats + max_move_power + ofensividad del tipo
features["offensive_raw"] = (
    features["total_base_stats"].fillna(0)
    + features["max_move_power"].fillna(0)
) * features["offensive_effectiveness"].fillna(1.0)

features[["id", "name", "offensive_effectiveness", "defensive_multiplier_mean", "offensive_raw", "defensive_raw"]].head()

,id,name,offensive_effectiveness,defensive_multiplier_mean,offensive_raw,defensive_raw
0,1,bulbasaur,0.964286,1.059524,432.000000,150.067416
1,2,ivysaur,0.964286,1.059524,515.892857,191.595506
2,3,venusaur,0.964286,1.059524,650.892857,248.224719
3,4,charmander,1.095238,1.000000,502.714286,132.000000
4,5,charmeleon,1.095238,1.000000,607.857143,181.000000


In [15]:
def min_max_norm(series):
    min_val = series.min()
    max_val = series.max()
    if pd.isna(min_val) or pd.isna(max_val) or min_val == max_val:
        return pd.Series(np.zeros(len(series)), index=series.index)
    return (series - min_val) / (max_val - min_val)

features["total_base_stats_norm"] = min_max_norm(features["total_base_stats"])
features["max_move_power_norm"] = min_max_norm(features["max_move_power"].fillna(0))
features["offensive_effectiveness_norm"] = min_max_norm(features["offensive_effectiveness"].fillna(1.0))
features["defensive_survivability_norm"] = min_max_norm(features["defensive_raw"].fillna(0))

features[[
    "id", "name",
    "total_base_stats_norm",
    "max_move_power_norm",
    "offensive_effectiveness_norm",
    "defensive_survivability_norm"
]].head()

,id,name,total_base_stats_norm,max_move_power_norm,offensive_effectiveness_norm,defensive_survivability_norm
0,1,bulbasaur,0.150526,0.52,0.25,0.131271
1,2,ivysaur,0.242105,0.52,0.25,0.191797
2,3,venusaur,0.368421,0.60,0.25,0.274333
3,4,charmander,0.141053,0.60,0.80,0.104938
4,5,charmeleon,0.242105,0.60,0.80,0.176355


In [16]:
out_path = DATA_GOLD / "pokemon_features.csv"
features.to_csv(out_path, index=False)
out_path, features.shape

(PosixPath('data/gold/pokemon_features.csv'), (1328, 28))

In [17]:
features[[
    "name", "type_1", "type_2",
    "total_base_stats",
    "max_move_power",
    "offensive_effectiveness",
    "defensive_multiplier_mean",
    "offensive_raw",
    "defensive_raw"
]].sort_values("total_base_stats", ascending=False).head(10)

,name,type_1,type_2,total_base_stats,max_move_power,offensive_effectiveness,defensive_multiplier_mean,offensive_raw,defensive_raw
1214,eternatus-eternamax,poison,dragon,1125,160.0,0.964286,1.011905,1239.107143,746.117647
1103,rayquaza-mega,dragon,flying,780,150.0,1.023810,1.107143,952.142857,275.483871
1068,mewtwo-mega-y,psychic,NaN,780,200.0,1.000000,1.095238,980.000000,270.260870
1067,mewtwo-mega-x,psychic,fighting,780,200.0,1.035714,1.095238,1015.000000,279.391304
1325,zygarde-mega,dragon,ground,778,NaN,1.059524,1.119048,824.309524,350.297872
1101,kyogre-primal,water,NaN,770,150.0,1.071429,1.000000,985.714286,350.000000
1102,groudon-primal,ground,fire,770,150.0,1.119048,1.023810,1029.523810,341.860465
1181,necrozma-ultra,psychic,dragon,754,160.0,0.988095,1.142857,903.119048,254.625000
492,arceus,normal,NaN,720,150.0,0.904762,1.000000,787.142857,360.000000
1144,zygarde-complete,dragon,ground,708,150.0,1.059524,1.119048,909.071429,386.042553
